# Creation of GDP and CPI Mimicking portfolios

## Data Prep

In [18]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [6]:
data = pd.read_csv('merged_data.csv', index_col=0)

In [13]:
data['def'] = np.log(data['corp_bond_return']/data['corp_bond_return'].shift(1)) - data['20y_return']
data['term'] = data['30y_return'] - data['rf']

In [67]:
data['defy'] = (data['BAA'] - data['AAA'])/100
data['termy'] = (data['10 year'] - data['1 year'])/100

In [104]:
# Predictors
predictors = sm.add_constant(data[['SL', 'SM', 'SH', 'BL', 'HM', 'BH',
                                   'def', 'term', 'defy', 'termy', 'rf']].copy()).dropna()
portfolios = ['SL', 'SM', 'SH', 'BL', 'HM', 'BH']
for portfolio in portfolios:
    predictors[portfolio] = predictors[portfolio]/100 - predictors['rf'] 

In [107]:
predictors

,const,SL,SM,SH,BL,HM,BH,def,term,defy,termy,rf
1973-04-30,1.0,-0.116366,-0.059144,-0.054957,-0.065703,-0.030393,-0.018819,-0.003262,-0.001740,0.0083,-0.0010,0.005002
1973-05-31,1.0,-0.111271,-0.077172,-0.078682,-0.013939,-0.025844,-0.038340,0.008574,-0.013672,0.0077,-0.0023,0.005098
1973-06-30,1.0,-0.056169,-0.034510,-0.032328,-0.014313,-0.008784,-0.014188,0.000121,-0.007495,0.0076,-0.0080,0.005379
1973-07-31,1.0,0.176029,0.095916,0.084404,0.057668,0.016414,0.043144,0.041822,-0.048362,0.0079,-0.0144,0.006540
1973-08-31,1.0,-0.045824,-0.059812,-0.047233,-0.039251,-0.036736,-0.013048,-0.032932,0.026077,0.0085,-0.0122,0.006855
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,1.0,0.001011,-0.024650,-0.024405,-0.049074,-0.022420,-0.017718,0.000711,-0.055391,0.0108,-0.0035,0.001901
2022-09-30,1.0,-0.095166,-0.102378,-0.099396,-0.093165,-0.092168,-0.087833,-0.030357,-0.087084,0.0110,-0.0022,0.001929
2022-10-31,1.0,0.070206,0.101226,0.136357,0.052402,0.105318,0.147344,0.014156,-0.067951,0.0116,-0.0056,0.002327
2022-11-30,1.0,-0.007285,0.027681,0.023308,0.048466,0.051688,0.045547,0.017402,0.075075,0.0117,-0.0106,0.002856


In [109]:
# GDP & CPI
gdp = data['BBKMGDP'].rename('gdp').to_frame().iloc[:-3]
cpi = data['CPIAUCSL'].rename('cpi').to_frame().iloc[:-3]

# Transformations 
gdp['gdp_mom'] = gdp['gdp'] / 100
gdp['level'] = (1 + gdp['gdp_mom']).cumprod()
gdp['gdp_yoy'] = np.log(gdp['level'] / gdp['level'].shift(12)) # 4 quarter change = 12 month change

cpi['cpi_mom'] = np.log(cpi['cpi'] / cpi['cpi'].shift(1))
cpi['cpi_yoy'] = np.log(cpi['cpi'] / cpi['cpi'].shift(12))

cpi = cpi.shift(1)

## GDP Factor

## CPI Factor